<a href="https://colab.research.google.com/github/Gargi030120/project1/blob/main/BrainTumorStageClassification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [2]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [5]:
base_dir = 'drive/MyDrive/archive/Training'
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [6]:
import os
data = []
labels = []

for category in categories:
    path = os.path.join(base_dir, category)
    class_num = categories.index(category)
    if not os.path.exists(path):
        print(f"Directory {path} does not exist!")
        continue
    class_num = categories.index(category)
    for img in os.listdir(path):
        try:
            img_array = load_img(os.path.join(path, img), target_size=(64, 64))  # Resize images
            img_array = img_to_array(img_array)
            data.append(img_array)
            labels.append(class_num)
        except Exception as e:
            print(f"Error loading image: {e}")

# Convert lists to numpy arrays
data = np.array(data, dtype='float32')
labels = np.array(labels)

In [7]:
data /= 255.0
from sklearn.model_selection import StratifiedShuffleSplit

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [8]:
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [9]:
# Standardize the data
scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

In [11]:
# Apply PCA
pca = PCA(n_components=50)  # Number of principal components to keep
X_train_pca = pca.fit_transform(X_train_flat)
X_test_pca = pca.transform(X_test_flat)

In [12]:
# Train SVM model
svm = SVC(kernel='linear', C=1)
svm.fit(X_train_pca, y_train)

SVC(C=1, kernel='linear')

In [13]:
# Predict on test data
y_pred = svm.predict(X_test_pca)

In [14]:
unique_classes=np.unique(y_test)
unique_classes_names=[categories[i] for i in unique_classes]

In [15]:
# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=unique_classes_names))

Classification Report:
              precision    recall  f1-score   support

      glioma       0.79      0.81      0.80       284
  meningioma       0.68      0.63      0.66       272
     notumor       0.86      0.86      0.86       294
   pituitary       0.89      0.92      0.90       295

    accuracy                           0.81      1145
   macro avg       0.80      0.81      0.80      1145
weighted avg       0.81      0.81      0.81      1145



In [16]:
print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

Accuracy Score:
0.8096069868995633


In [18]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def preprocess_image(image_path, target_size=(64, 64)):
    """Load and preprocess a single image."""
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    img_array = img_array.reshape(1, -1)  # Flatten and reshape for the model
    img_array = scaler.transform(img_array)  # Apply the same scaler used during training
    img_array = pca.transform(img_array)  # Apply PCA transformation
    return img_array

def predict_image(model, image_array, categories):
    """Predict the class of a single image."""
    prediction = model.predict(image_array)
    class_idx = prediction[0]
    return categories[class_idx]

# Path to the image you want to test
test_image_path = 'drive/MyDrive/archive/Training/meningioma/Tr-meTr_0003.jpg'

# Preprocess the image
test_image_array = preprocess_image(test_image_path)

# Predict the class of the image
predicted_class = predict_image(svm, test_image_array, categories)

print(f"The predicted class for the test image is: {predicted_class}")

The predicted class for the test image is: meningioma
